In [2]:
import importlib
import os
import sys
import warnings

import numpy as np

from music21 import environment

environment.set('midiPath', '/usr/bin/timidity')

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# Всё, что выводит CUDA / XLA, будет записано в файл
sys.stderr = open("xla_warnings.log", "w")



sys.path.append('src')

# в режиме редактирования сбрасуем кэшь иначе не видно изменений
for module in [
    "utils.music21_utils",
    "utils.data_utils",
    "utils.model_utils",
    "utils.generate_utils",
    "utils.train_utils",
    "utils.__init__"
    
]:
    imported_module = importlib.import_module(module)
    importlib.reload(imported_module)


from utils.__init__ import (
    midi_to_notes, split_parts, combine_to_score,
    normalize_sequence, denormalize_sequence, prepare_decoder_inputs,
    build_transformer,
    generate_autoregressive, generated_to_part,
    train_model,
    get_midis_by_compositor,
    get_notes_multitrack,
    prepare_sequences
)

2025-11-14 23:42:25.950416: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763152945.963786  119857 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763152945.968008  119857 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763152945.977984  119857 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763152945.977996  119857 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763152945.977997  119857 computation_placer.cc:177] computation placer alr

## Загрузка midi по композитору

In [3]:
сhopin_midis = get_midis_by_compositor("Chopin")

In [4]:
# все ноты
notes = get_notes_multitrack(сhopin_midis)
print(f"Извлечено {len(notes)} нот из {len(сhopin_midis)} файлов")

Извлечено 57894 нот из 48 файлов


In [5]:
# --- Разделяем инструменты ---
melody_part, bass_part, chord_part = split_parts(notes)

In [6]:
# 2. Преобразование
# melody_X = normalize_sequence([[n['pitch'], n['step'], n['duration']] for n in melody_part])[None, ...]
chord_y = normalize_sequence([[n['pitch'], n['step'], n['duration']] for n in chord_part])[None, ...]
dec_inputs, dec_targets = prepare_decoder_inputs(chord_y)

In [9]:
# Преобразуем партии в массивы признаков
melody_X, melody_y = prepare_sequences(melody_part)
bass_X, bass_y = prepare_sequences(bass_part)

In [10]:
# chord_part — это список словарей
chord_seq = [[n['pitch'], float(n['step']), float(n['duration'])] for n in chord_part]

In [11]:
chord_X, chord_y = prepare_sequences(chord_seq)

In [12]:
# --- Синхронизируем длины ---
min_len = min(len(melody_X), len(bass_X), len(chord_X))
melody_X, melody_y = melody_X[:min_len], melody_y[:min_len]
bass_X, bass_y = bass_X[:min_len], bass_y[:min_len]
chord_X, chord_y = chord_X[:min_len], chord_y[:min_len]

In [13]:
# вариант: конкат по времени (length = 2 * L_enc)
# --- Входы и выходы для трансформера ---
enc_inputs = np.concatenate([melody_X, bass_X], axis=1)
dec_inputs = np.zeros_like(chord_y)
dec_inputs[:, 1:, :] = chord_y[:, :-1, :]  # shift right
dec_targets = chord_y

## Обучение

In [18]:
# 3. Модель
model = build_transformer()
train_model(model, enc_inputs, dec_inputs, dec_targets, epochs=10)

Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 97s 427ms/step - loss: 0.3171 - val_loss: 0.0047
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0215 - val_loss: 0.0035
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0110 - val_loss: 0.0032
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0083 - val_loss: 0.0037
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0072 - val_loss: 0.0032
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0066 - val_loss: 0.0033
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0063 - val_loss: 0.0033
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0061 - val_loss: 0.0033
Epoch 9/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.0060 - val_loss: 0.0035
Epoch 10/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0059 - val_loss: 0.0034


## Генерация

In [19]:
# 4. Генерация
gen = generate_autoregressive(model, melody_X, dec_inputs[:, :4, :], length=32)
gen_denorm = denormalize_sequence(gen)
part = generated_to_part(gen_denorm, make_chords=True)

In [16]:
combine_to_score([part]).show('midi')

Playing /tmp/tmpr73fh5i1.mid
MIDI file: /tmp/tmpr73fh5i1.mid
Format: 1  Tracks: 2  Divisions: 10080
Track name: Piano
Playing time: ~18 seconds
Notes cut: 0
Notes lost totally: 0


In [20]:
combine_to_score([part]).write('midi', fp='chopin_rest_by_first_transformer.midi')

'chopin_rest_by_first_transformer.midi'